In [1]:
deployment = "gpt4"
model = "gpt-4"

# 回顾

In [3]:
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain.chains import LLMRequestsChain
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(deployment_name = deployment, model_name=model, temperature=0, max_tokens=200)

# https://www.cnblogs.com/sunsky303/p/11045113.html

def query_web(question):
      print("http://cn.bing.com/search?q=" + question.replace(" ", "+").replace("?",""))
      template = """Between >>> and <<< are the raw search result text from google.
      Extract the answer to the question '{query}' or say "not found" if the information is not contained.
      Use the format
      Extracted:<answer or "not found">
      >>> {requests_result} <<<
      Extracted:"""

      PROMPT = PromptTemplate(
          input_variables=["query", "requests_result"],
          template=template,
      )

      inputs = {
          "query": question,
          "url": "https://cn.bing.com/search?q=" + question.replace(" ", "+").replace("?","")
      }
      requests_chain = LLMRequestsChain(llm_chain = LLMChain(llm=llm, prompt=PROMPT), output_key="query_info")
      res = requests_chain.run(inputs)
      print(f"Result:{res}")
      return res

In [4]:
def get_current_cluster_state(cluster_name):
    return  """ERROR: Failed to pull image "chaocai/docker/dsp:latest"""

In [5]:
funcs = {
    "get_current_cluster_state": get_current_cluster_state,
    "query_web":query_web,
}

In [52]:
import openai
import json
openai.api_version = "2023-07-01-preview"
def run(input):
    msg=[{"role":"user","content":input}]
    ret = run_conversation(msg)
    return ret["content"] 

def run_conversation(msg):
    response = openai.ChatCompletion.create(
        engine=deployment,
        model=model,
        messages=msg,
        functions=[
     
            {
                "name": "get_current_cluster_state",
                "description": "Get the current state in a given cluster",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "cluster_name": {
                            "type": "string",
                            "description": "the name of the cluster",
                        },
                        
                    },
                    "required": ["cluster_name"],
                },
            },
     
            {
                "name": "query_web",
                "description": "Get a realtime info/unknown info from web",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                            "description": "the query/question",
                        },
                        
                    },
                    "required": ["query"],
                },
            }
        ],
        function_call="auto",
    )

    message =  response["choices"][0]["message"]
    
    # 如果不需要调用function，则直接返回结果
    if not message.get("function_call"):
        return message
    
    # 从
    function_name = message["function_call"]["name"]
    function_args = json.loads(message["function_call"]["arguments"])
    print(function_args)
    res = funcs[function_name](function_args["question"])
    message["content"]=None
    msg.append(message)
    msg.append({
                "role": "function",
                "name": function_name,
                "content": res,
            })
    return run_conversation(msg)

In [55]:
run("北京今天适合穿什么")

{'question': '北京今天的天气如何'}
http://cn.bing.com/search?q=北京今天的天气如何
Result:今天白天晴转多云,傍晚山区有雷阵雨,北转南风二三级,最高气温34℃;夜间多云转晴,南转北风一二级,最低气温24℃。


'今天北京的气温在24℃-34℃之间，天气晴转多云，傍晚山区有雷阵雨。因此，建议您穿上短袖和裤子，携带一把雨伞以防雷阵雨，保持身体的干爽。晚上气温稍低，可能需要携带一件轻薄夹克或者开衫防止凉。'

# LangChain Agent

In [6]:
from langchain.agents import initialize_agent, Tool
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import LLMRequestsChain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import time


In [7]:
tools = [
    Tool(
        name = "Get current info",func=query_web,
        description="""invoke it for when you need to answer questions about current info.
             And the input should be a search query."""),
    Tool(
        name="Get current cluster state", func=get_current_cluster_state,
        description="""invoke it for when you need to get the current state in a given cluster, 
        And the input should be service name.""")
]

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", max_iterations=3, verbose=True)

In [8]:
agent.run("北京今天穿什么合适？")



> Entering new AgentExecutor chain...
我需要获取当前的天气信息来回答这个问题。
Action: Get current info
Action Input: 北京今天的天气http://cn.bing.com/search?q=北京今天的天气
Result:晴 34 / 23℃

Observation: 晴 34 / 23℃
Thought:根据观察到的天气信息，今天北京的天气是晴朗，最高温度34度，最低温度23度。
Final Answer: 今天北京的天气比较热，建议穿短袖、短裤或者薄的长袖，同时带上防晒霜和遮阳帽。

> Finished chain.


'今天北京的天气比较热，建议穿短袖、短裤或者薄的长袖，同时带上防晒霜和遮阳帽。'

In [11]:
agent.run("如何修复的ADX集群？")



> Entering new AgentExecutor chain...
首先，我需要了解ADX集群的当前状态。
Action: Get current cluster state
Action Input: ADX集群
Observation: ERROR: Failed to pull image "chaocai/docker/dsp:latest
Thought:看来ADX集群的问题在于无法拉取最新的docker镜像。我需要查找如何解决这个问题。
Action: Get current info
Action Input: 如何修复无法拉取docker镜像的问题Question:如何修复无法拉取docker镜像的问题
Result:not found

Observation: not found
Thought:我没有找到具体的解决方案，但通常，无法拉取docker镜像的问题可能是由于网络问题或者docker仓库的问题。我可以建议一些可能的解决步骤。
Final Answer: 无法拉取docker镜像的问题可能是由于网络问题或者docker仓库的问题。你可以尝试以下步骤来解决这个问题：
1. 检查你的网络连接。
2. 确保你有权限访问docker仓库。
3. 检查docker仓库是否存在问题或者是否有最新的镜像。
4. 如果以上步骤都无法解决问题，你可能需要联系docker仓库的管理员或者寻求专业的技术支持。

> Finished chain.


'无法拉取docker镜像的问题可能是由于网络问题或者docker仓库的问题。你可以尝试以下步骤来解决这个问题：\n1. 检查你的网络连接。\n2. 确保你有权限访问docker仓库。\n3. 检查docker仓库是否存在问题或者是否有最新的镜像。\n4. 如果以上步骤都无法解决问题，你可能需要联系docker仓库的管理员或者寻求专业的技术支持。'

# 利用LangChain中的工具

In [7]:
pip install wolframalpha

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 548.8 kB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [61]:
from langchain.utilities.wolfram_alpha import WolframAlphaAPIWrapper
def wolfram_api(question):
    wolfram = WolframAlphaAPIWrapper()
    return wolfram.run(question)

In [62]:
wolfram_api("What is 2x+5 = -3x + 7?")

'Assumption: 2 x + 5 = -3 x + 7 \nAnswer: x = 2/5'

In [63]:
tools = [
    Tool(
        name = "Get current info",func=query_web,
        description="""invoke it for when you need to answer questions about current info.
             And the input should be a search query."""),
    Tool(
        name="Get current cluster state", func=get_current_cluster_state,
        description="""invoke it for when you need to get the current state in a given cluster, 
        And the input should be service name."""),
    Tool(
        name="Invoke WolframAlpha API", func=wolfram_api,
        description="""invoke it for when you need to anwser or solve a mathematical question, 
        And the input is the question""")
]

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", max_iterations=3, verbose=True)

In [13]:
agent.run("在一个外星球上，这里的鸡都长着3只脚1个头，免子有4只脚1个头，一个装着鸡和免子的笼子里一共有20个头，71只脚，请问笼子里各有多少只鸡和免子？")



> Entering new AgentExecutor chain...
这是一个数学问题，可以通过设置方程来解决。设鸡的数量为x，免子的数量为y。那么我们可以得到两个方程：x + y = 20 (头的数量) 和 3x + 4y = 71 (脚的数量)。我可以使用WolframAlpha API来解决这个问题。
Action: Invoke WolframAlpha API
Action Input: Solve the equations x + y = 20 and 3x + 4y = 71
Observation: Assumption: solve x + y = 20
3 x + 4 y = 71 
Answer: x = 9 and y = 11
Thought:我现在知道了最后的答案
Final Answer: 笼子里有9只鸡和11只免子。

> Finished chain.


'笼子里有9只鸡和11只免子。'